# Visual intuition for Cross Product, Skew Matrices, and Quaternions (Basics)
This notebook is meant to be *played with*: tweak vectors, rerun cells, and watch how the geometry changes.

**Topics:**
- Cross product as perpendicular vector + area
- Cross product matrix (skew-symmetric / hat operator)
- Angular velocity:  $\dot v = \omega \times v$
- Quaternion basics: why 4 numbers, multiplication, inverse
- Quaternion rotation vs Euler angles
- How dot+cross appear inside quaternion multiplication

> Tip: if 3D plots look tiny, use the toolbar “zoom” and rotate with your mouse.

In [ ]:
# --- Imports ---
import numpy as np
import matplotlib.pyplot as plt

# 3D plotting
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

np.set_printoptions(precision=4, suppress=True)

## Helper functions for plotting vectors in 3D

In [ ]:
def setup_3d(ax, lim=1.5, title=None):
    ax.set_xlim([-lim, lim]); ax.set_ylim([-lim, lim]); ax.set_zlim([-lim, lim])
    ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('z')
    ax.set_box_aspect([1,1,1])
    if title:
        ax.set_title(title)

def draw_vec(ax, v, label=None, origin=(0,0,0), linewidth=2):
    v = np.asarray(v).reshape(3,)
    ox, oy, oz = origin
    ax.quiver(ox, oy, oz, v[0], v[1], v[2], arrow_length_ratio=0.08, linewidth=linewidth)
    if label:
        ax.text(ox+v[0], oy+v[1], oz+v[2], f" {label}", fontsize=10)

def norm(v): 
    v = np.asarray(v)
    return float(np.linalg.norm(v))

def unit(v, eps=1e-12):
    v = np.asarray(v, dtype=float)
    n = np.linalg.norm(v)
    if n < eps:
        return v*0.0
    return v/n

## 1) Cross product: direction + area
For vectors **a** and **b**:
- **Direction**: $a\times b$ is perpendicular to the plane spanned by $a,b$ (right-hand rule)
- **Magnitude**: $\|a\times b\| = \|a\|\|b\|\sin\theta$ = area of the parallelogram


In [ ]:
# Try changing these:
a = np.array([1.2, 0.2, 0.0])
b = np.array([0.4, 1.0, 0.3])

c = np.cross(a, b)

theta = np.arccos(np.clip(np.dot(a,b)/(norm(a)*norm(b)+1e-12), -1, 1))
area = norm(c)

print("a =", a)
print("b =", b)
print("a×b =", c)
print("theta (deg) =", float(np.degrees(theta)))
print("||a×b|| (area) =", area)

fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection='3d')
setup_3d(ax, lim=max(1.6, norm(a), norm(b), norm(c))*1.1, title="Cross product: a, b, and a×b")

draw_vec(ax, a, "a")
draw_vec(ax, b, "b")
draw_vec(ax, c, "a×b")

# draw parallelogram edges (thin lines)
O = np.zeros(3)
A = a; B = b; AplusB = a + b
ax.plot([O[0], A[0]], [O[1], A[1]], [O[2], A[2]], linewidth=1)
ax.plot([O[0], B[0]], [O[1], B[1]], [O[2], B[2]], linewidth=1)
ax.plot([A[0], AplusB[0]], [A[1], AplusB[1]], [A[2], AplusB[2]], linewidth=1)
ax.plot([B[0], AplusB[0]], [B[1], AplusB[1]], [B[2], AplusB[2]], linewidth=1)

plt.show()

### Quick sanity checks
- If **a** and **b** are parallel, $a\times b=0$.
- Swapping order flips sign: $b\times a = -(a\times b)$.


In [ ]:
a2 = np.array([1,0,0])
b2 = np.array([2,0,0])   # parallel
print("parallel example:", np.cross(a2,b2))

print("swap sign example:")
print("a×b =", np.cross(a,b))
print("b×a =", np.cross(b,a))

## 2) Cross product as a matrix multiplication: the skew-symmetric matrix
Define
$$
[x] = \begin{bmatrix}
0 & -x_3 & x_2\\
x_3 & 0 & -x_1\\
-x_2 & x_1 & 0
\end{bmatrix}
$$
Then $x\times y = [x]y$. Also $[x]^T = -[x]$ (skew-symmetric).


In [ ]:
def skew(x):
    x1,x2,x3 = np.asarray(x).reshape(3,)
    return np.array([[0, -x3, x2],
                     [x3, 0, -x1],
                     [-x2, x1, 0]], dtype=float)

x = np.array([0.2, -0.6, 1.0])
y = np.array([1.1,  0.4, 0.2])

lhs = np.cross(x,y)
rhs = skew(x) @ y

print("x×y =", lhs)
print("[x]y =", rhs)
print("difference =", lhs-rhs)
print("[x]^T + [x] =
", skew(x).T + skew(x))

## 3) Angular velocity and why $\dot v = \omega \times v$
If a vector **v** is rigidly attached to a rotating body, its time derivative in a fixed frame is:
$$\dot v = \omega \times v$$
This matches the intuition: the derivative is perpendicular to **v** (rotation changes direction, not length).


In [ ]:
# Example: constant angular velocity about z-axis
omega = np.array([0.0, 0.0, 1.5])  # rad/s
v0 = np.array([1.0, 0.0, 0.0])

dt = 0.02
T = 3.0
ts = np.arange(0, T+dt, dt)

# Integrate dv/dt = omega × v (simple Euler integration for intuition)
v = v0.copy()
traj = [v.copy()]
for _ in ts[1:]:
    v = v + dt*np.cross(omega, v)
    v = unit(v)  # keep unit length for clarity
    traj.append(v.copy())
traj = np.array(traj)

fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection='3d')
setup_3d(ax, lim=1.3, title=r"Trajectory from $\dot v = \omega 	imes v$ (about z-axis)")

draw_vec(ax, omega, r"$\omega$")
ax.plot(traj[:,0], traj[:,1], traj[:,2])
draw_vec(ax, v0, "v(0)")
draw_vec(ax, traj[-1], "v(T)")

plt.show()

## 4) Quaternion basics (what you actually use)
A quaternion is $q = (w, x, y, z)$ (scalar + 3-vector). For rotations we use **unit quaternions**: $\|q\|=1$.

### Why 4 not 3? (intuition)
- In 3D, a number-like multiplication with identity + inverses for all nonzero elements runs into fundamental obstacles.
- In 4D, quaternions give a norm-compatible multiplication, so every nonzero quaternion has an inverse.

When you split quaternion multiplication into scalar/vector parts, you see dot and cross products:
$$
(a_0, a)(b_0, b) = (a_0 b_0 - a\cdot b,\; a_0 b + b_0 a + a\times b)
$$
That extra scalar slot is the “missing 1D piece” that makes multiplication invertible and well-behaved.


### Minimal quaternion utilities

In [ ]:
def q_mul(q1, q2):
    # q = (w, x, y, z)
    w1,x1,y1,z1 = q1
    w2,x2,y2,z2 = q2
    return np.array([
        w1*w2 - x1*x2 - y1*y2 - z1*z2,
        w1*x2 + x1*w2 + y1*z2 - z1*y2,
        w1*y2 - x1*z2 + y1*w2 + z1*x2,
        w1*z2 + x1*y2 - y1*x2 + z1*w2
    ], dtype=float)

def q_conj(q):
    w,x,y,z = q
    return np.array([w, -x, -y, -z], dtype=float)

def q_norm(q):
    return float(np.linalg.norm(q))

def q_inv(q, eps=1e-12):
    n2 = q_norm(q)**2
    if n2 < eps:
        raise ValueError("Quaternion norm too small for inverse.")
    return q_conj(q)/n2

def q_unit(q, eps=1e-12):
    n = q_norm(q)
    if n < eps:
        raise ValueError("Quaternion norm too small to normalize.")
    return q/n

def q_from_axis_angle(axis, angle_rad):
    axis = unit(axis)
    half = angle_rad/2.0
    return q_unit(np.array([np.cos(half), *(np.sin(half)*axis)], dtype=float))

def q_rotate_vec(q, v):
    # v' = q (0,v) q^{-1}; for unit q, inverse = conjugate
    q = q_unit(q)
    vq = np.array([0.0, *v], dtype=float)
    return q_mul(q_mul(q, vq), q_conj(q))[1:]

## 5) Visualizing quaternion rotation
Axis-angle $(\hat u,\theta)$ maps to unit quaternion:
$$q = \left(\cos\frac{\theta}{2},\; \hat u\sin\frac{\theta}{2}\right)$$
Rotate a vector by $v' = q(0,v)q^{-1}$.


In [ ]:
axis = np.array([0.3, 0.8, 0.5])
theta = np.deg2rad(80)

q = q_from_axis_angle(axis, theta)

v = np.array([1.0, 0.2, 0.0])
v_rot = q_rotate_vec(q, v)

print("axis =", unit(axis))
print("theta (deg) =", np.degrees(theta))
print("q =", q, " (unit norm =", q_norm(q), ")")
print("v =", v)
print("v_rot =", v_rot)

fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection='3d')
setup_3d(ax, lim=1.5, title="Quaternion rotation of a vector")

draw_vec(ax, unit(axis), "axis (unit)")
draw_vec(ax, v, "v")
draw_vec(ax, v_rot, "R_q v")

# arc-like samples
ts = np.linspace(0, theta, 60)
curve = np.array([q_rotate_vec(q_from_axis_angle(axis, t), v) for t in ts])
ax.plot(curve[:,0], curve[:,1], curve[:,2])

plt.show()

## 6) Dot + cross appear inside quaternion multiplication
Let pure-vector quaternions be $p=(0,a)$ and $r=(0,b)$. Then:
$$pr = (-a\cdot b,\; a\times b)$$
So scalar part is **minus dot**, vector part is **cross**.


In [ ]:
a = np.array([0.6, -0.2, 0.8])
b = np.array([0.3,  0.9, 0.1])

p = np.array([0.0, *a])
r = np.array([0.0, *b])

pr = q_mul(p, r)

print("a·b =", np.dot(a,b))
print("a×b =", np.cross(a,b))
print("p*r =", pr, " = (scalar, vector)")
print("scalar part (should be -a·b):", pr[0], "vs", -np.dot(a,b))
print("vector part (should be a×b):", pr[1:], "vs", np.cross(a,b))

## 7) Quaternion composition demo (many small rotations)
This simulates composing incremental rotations (like integrating angular velocity).
We track the final body axes (columns of $R$) and the final quaternion.


In [ ]:
def R_from_axis_angle(axis, angle):
    axis = unit(axis)
    x,y,z = axis
    c = np.cos(angle); s = np.sin(angle); C = 1-c
    return np.array([
        [c + x*x*C,     x*y*C - z*s, x*z*C + y*s],
        [y*x*C + z*s,   c + y*y*C,   y*z*C - x*s],
        [z*x*C - y*s,   z*y*C + x*s, c + z*z*C]
    ], dtype=float)

np.random.seed(0)
N = 120
dt = 0.03

omegas = []
for k in range(N):
    w = np.array([0.8*np.sin(0.05*k), 0.9*np.cos(0.04*k), 0.4*np.sin(0.03*k)])
    omegas.append(w)
omegas = np.array(omegas)

R = np.eye(3)
q = np.array([1.0, 0.0, 0.0, 0.0])

for w in omegas:
    angle = norm(w)*dt
    if angle < 1e-12:
        dR = np.eye(3)
        dq = np.array([1.0, 0, 0, 0], dtype=float)
    else:
        axis = w/(norm(w)+1e-12)
        dR = R_from_axis_angle(axis, angle)
        dq = q_from_axis_angle(axis, angle)

    # Convention: apply incremental rotation in the space frame (left multiply)
    R = dR @ R
    q = q_mul(dq, q)
    q = q_unit(q)

R_final = R
x_b, y_b, z_b = R_final[:,0], R_final[:,1], R_final[:,2]

fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection='3d')
setup_3d(ax, lim=1.2, title="Final body axes (columns of R) after many small rotations")

draw_vec(ax, x_b, r"$\hat x_b$")
draw_vec(ax, y_b, r"$\hat y_b$")
draw_vec(ax, z_b, r"$\hat z_b$")

draw_vec(ax, [1,0,0], "x_s", linewidth=1)
draw_vec(ax, [0,1,0], "y_s", linewidth=1)
draw_vec(ax, [0,0,1], "z_s", linewidth=1)

plt.show()

print("Final rotation matrix R:\n", R_final)
print("Final quaternion q (unit):\n", q, "norm =", q_norm(q))

## 8) One-page conceptual wrap-up: why 4D fixes what 3D can't
- Cross product is great geometry, but it is not a division-friendly multiplication (zero divisors, no identity, not associative).
- Quaternions add a scalar part and give a norm-compatible multiplication.
- In quaternion multiplication, dot and cross show up automatically.

If you want the optional theorem statement later: the real normed division algebras exist only in dimensions 1, 2, 4, 8.


## Exercises (optional)
1) Edit **a** and **b** in Section 1 until $a\times b\approx 0$. What geometric condition did you create?
2) In Section 5, change the axis and angle. Does the arc direction match the right-hand rule?
3) In Section 6, set **a=b**. What happens to the vector part? Why?
4) In Section 3, increase $\|\omega\|$ and/or dt. When does the simple Euler integration look wrong?
